In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 50.9 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload dataset.json

Saving iot_dataset.json to iot_dataset.json


In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="iot_dataset.json", field=None)["train"]
print(dataset.column_names)


Generating train split: 0 examples [00:00, ? examples/s]

['text']


In [ ]:
from datasets import Dataset

# Load JSON
import json
with open("iot_dataset.json", "r") as f:
    raw_data = json.load(f)

# Split 'text' into input_text (prompt) and target_text (summary)
def split_text(item):
    full_text = item["text"]
    # Split at 'Generate IoT monitoring summary:' which is the separator
    if "Generate IoT monitoring summary:" in full_text:
        prompt, response = full_text.split("Generate IoT monitoring summary:", 1)
        return {"input_text": prompt.strip(), "target_text": response.strip()}
    else:
        # fallback if split fails
        return {"input_text": full_text, "target_text": full_text}

dataset = [split_text(x) for x in raw_data]

# Convert to HF Dataset
hf_dataset = Dataset.from_list(dataset)
print(hf_dataset.column_names)
# Output should now be ['input_text', 'target_text']


['input_text', 'target_text']


In [ ]:
from datasets import load_dataset, Dataset

# Step 1: Load your JSON dataset
dataset = load_dataset("json", data_files="iot_dataset.json")["train"]
print(dataset.column_names)  # Should print ['text']

# Step 2: Split 'text' into input and target
def split_text(example):
    full_text = example["text"]
    if "Generate IoT monitoring summary:" in full_text:
        prompt, summary = full_text.split("Generate IoT monitoring summary:", 1)
        return {"input_text": prompt.strip(), "target_text": summary.strip()}
    else:
        return {"input_text": full_text, "target_text": full_text}

dataset = dataset.map(split_text, remove_columns=["text"])
print(dataset.column_names)  # Now should print ['input_text', 'target_text']

# Step 3: Tokenize
from transformers import AutoTokenizer

MODEL_NAME = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize(batch):
    inputs = tokenizer(batch["input_text"], truncation=True, padding="max_length", max_length=512)
    targets = tokenizer(batch["target_text"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

print(tokenized_dataset[0])


Generating train split: 0 examples [00:00, ? examples/s]

['text']


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

['input_text', 'target_text']


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'input_ids': tensor([    0, 39525,    35,   291,  2481,    12,  4197,    12,  1225, 50118,
        42586, 18205,    35,   231, 50118, 37591,  1163,  1033,    35, 37988,
        50118, 46315, 36634,    35,  2908,     4,   246, 50118, 47239, 13634,
           35,   132, 50118, 43370, 13634,    35,   155,     2,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1, 

In [ ]:
# 1️⃣ Install required packages (if not already)
#!pip install -q transformers datasets peft accelerate bitsandbytes

# 2️⃣ Imports
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model

# -----------------------
# 3️⃣ Load base model & tokenizer
# -----------------------
MODEL_NAME = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# -----------------------
# 4️⃣ Apply LoRA
# -----------------------
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],  # BART attention layers
    lora_dropout=0.05,
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Confirm only LoRA params are trainable


Loading weights:   0%|          | 0/511 [00:00<?, ?it/s]

trainable params: 1,179,648 || all params: 407,470,080 || trainable%: 0.2895


In [ ]:
from peft import LoraConfig, get_peft_model

# -----------------------
# Step 3: Apply LoRA
# -----------------------
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],  # BART attention layers
    lora_dropout=0.05,
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Confirm only LoRA params are trainable


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 1,179,648 || all params: 407,470,080 || trainable%: 0.2895


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./bart_iot_lora",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    fp16=True,               # use fp16 on GPU, disable if TPU
    predict_with_generate=True
)


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    # tokenizer=tokenizer  <-- remove this line
)


In [ ]:
trainer.train()

Step,Training Loss
10,10.655499
20,10.391266
30,9.990441
40,9.625903
50,9.295332
60,9.020094
70,8.658465
80,8.326374
90,7.882471
100,7.417133


TrainOutput(global_step=1500, training_loss=1.9769712969462077, metrics={'train_runtime': 237.5176, 'train_samples_per_second': 12.631, 'train_steps_per_second': 6.315, 'total_flos': 3261528539136000.0, 'train_loss': 1.9769712969462077, 'epoch': 3.0})

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained("./bart_iot_lora_model")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
model = PeftModel.from_pretrained(model, "./bart_iot_lora_model")


Loading weights:   0%|          | 0/511 [00:00<?, ?it/s]

In [ ]:
inputs = tokenizer(
    "Date: 2026-02-19\nActive Devices: 5\nTotal Readings: 300\nAverage Temperature: 38.2\nCritical Events: 1\nWarning Events: 2\n\nGenerate IoT monitoring summary:",
    return_tensors="pt"
)

outputs = model.generate(**inputs, max_length=100)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)


🔴 CRITICAL: 1 devices exceeded safe limits. Average temperature was 38.2°C with 300 readings over 300°C. The average range was 38°C to 38°D with 200°C within range. Cyclic events occurred 1 warning events were detected.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel
import random
from datetime import datetime, timedelta
import torch

# -----------------------
# Load LoRA-fine-tuned BART
base_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = PeftModel.from_pretrained(base_model, "./bart_iot_lora_model")
model.eval()

# -----------------------
# Function to generate test prompts
def generate_iot_prompt():
    date = datetime.now() - timedelta(days=random.randint(0, 30))
    date_str = date.strftime("%Y-%m-%d")
    active_devices = random.randint(2, 8)
    total_readings = random.randint(100, 500)
    avg_temp = round(random.uniform(25, 45), 1)
    critical = random.choice([0, 0, 1, 2])
    warning = random.randint(0, 3)

    prompt = f"""
Date: {date_str}
Active Devices: {active_devices}
Total Readings: {total_readings}
Average Temperature: {avg_temp}
Critical Events: {critical}
Warning Events: {warning}

Generate IoT monitoring summary:
"""
    return prompt.strip()

# -----------------------
# Test multiple inputs
for i in range(5):  # generate 5 test summaries
    test_text = generate_iot_prompt()
    inputs = tokenizer(test_text, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=100)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"Test Input {i+1}:\n{test_text}\n")
    print(f"Generated Summary:\n{summary}\n")
    print("------\n")


Loading weights:   0%|          | 0/511 [00:00<?, ?it/s]

Test Input 1:
Date: 2026-01-30
Active Devices: 2
Total Readings: 112
Average Temperature: 39.8
Critical Events: 0
Warning Events: 1

Generate IoT monitoring summary:

Generated Summary:
🔴 CRITICAL: 1 warning events detected. Average temperature 39.8°C with 112 readings per square centimetre. WARNING: 1 devices exceeded safe limits. WARNING GRAPHIC CONTENT: Despite monitoring success, average temperatures exceeded safe ranges.

------

Test Input 2:
Date: 2026-02-15
Active Devices: 7
Total Readings: 152
Average Temperature: 38.5
Critical Events: 0
Warning Events: 3

Generate IoT monitoring summary:

Generated Summary:
🔴 CRITICAL: 3 warning events detected. Average temperature 38.5°C with 152 readings on average. Average temperatures range from 38°C to 50°C. The average range is higher than normal. WARNING: Warning events detected three devices exceeding safe limits.

------

Test Input 3:
Date: 2026-02-18
Active Devices: 2
Total Readings: 147
Average Temperature: 29.4
Critical Events: 0